# Single state methods

Suppose that we have the following maximization problem:

$$
    \underset{x \in \mathbb{S}}{\operatorname{\argmax}} \; f(x)
$$

where $f$ is called fitenss function and measures the "quality" of a solution.

It could be very difficult to find the optimal solution $x_{optimal}$.

Now, suppose that the set of solutions $\mathbb{S}$ is not empty and let $x_1$ be a sub-optimal solutions such that $f(x_1) \leq f(x_{optimal})$. Our goal is to improve our candidate solution, i.e. minimize the distance between $f(x_{candidate})$ and $f(x_{optimal})$.

The following problem is a trivial problem but it is usefull to introduce concepts and methods about optimization.

## OneMax problem
Let $\mathbb{S} = \{0,1\}^n$, the size of the space is $2^n$.

Let $x \in \mathbb{S}$, the function $f$ is defined as the number of ones contained in x.

Clearly, $$x_{optimal} = 1^n$$ 


In [41]:
import random

class OneMaxProblem:
    '''
        A simple class for the OneMax problem.
    '''
    def __init__(self, n):
        self.n = n

    def fitness(self, x):
        return len([digit for digit in x if digit == 1])
    
problem = OneMaxProblem(n=4)
x = [1, 0, 1, 1]
print(f"The fitness of x={x} is {problem.fitness(x)}.")


The fitness of x=[1, 0, 1, 1] is 3.


## Random Search

Random Search method is based on generating randomly new solutions and then it returns the best one.

Let's add a method to the OneMaxProblem class in order to generate a random binary list of fixed length.

In [42]:
def random_solution(self):
    return [random.randint(0, 1) for _ in range(self.n)]

OneMaxProblem.random_solution = random_solution

In [43]:
def random_search(problem, max_evaluations):
    '''
        A simple random search algorithm for the OneMax problem.
        The termination condition is the number of evaluations.
    '''
    best_x = None # best solution found so far
    best_fitness = -1 # fitness of the best solution found
    evaluations = 0 # number of solutions evaluated
    while evaluations < max_evaluations: # termination condition
        x = problem.random_solution() # generate a random solution
        x_fitness = problem.fitness(x) # evaluate the solution
        evaluations += 1 # increase the counter
        if x_fitness > best_fitness: # if we found a better solution
            best_x = x.copy() # update the best solution
            best_fitness = x_fitness # update the fitness of the best solution
    
    return (best_x, best_fitness)

In [44]:
random_search(problem, max_evaluations=4)

([0, 1, 1, 1], 3)

## Hill climbing

Hill climbing method is based on the concept of neighbor of a solution.
The neighborhood of a binary string $x$ is given by all the strings that are at Hamming distance one from $x$. For example, the neighborhood of $110$ is: $\{010, 100,111\}$.

Recall that the Hamming distance of two strings is the number of positions at which the corrisponding symbols differ.

In [45]:
def random_neighbor(self, x):
    '''
        A function that returns a random neighbor of a solution x at Hamming distance 1.
    '''
    x_new = x.copy() # copy the solution
    i = random.randint(0, self.n-1) # select a random position
    x_new[i] = 1 - x_new[i] # flip the i-th bit
    return x_new

OneMaxProblem.random_neighbor = random_neighbor

In [46]:
def hill_climbing(problem, max_evaluations):
    '''
        A simple hill climbing algorithm for the OneMax problem.
        The termination condition is the number of evaluations.
    '''
    best_x = problem.random_solution() # generate a random solution
    best_fitness = problem.fitness(best_x) # evaluate the solution
    evaluations = 1 # number of solutions evaluated
    while evaluations < max_evaluations: # termination condition
        x_neighbor = problem.random_neighbor(best_x) # generate a random neighbor
        x_neighbor_fitness = problem.fitness(x_neighbor) # evaluate the neighbor
        evaluations += 1 # increase the counter
        if x_neighbor_fitness > best_fitness: # if we found a better solution
            best_x = x_neighbor.copy() # update the best solution
            best_fitness = x_neighbor_fitness # update the fitness of the best solution
    
    return (best_x, best_fitness)

In [47]:
hill_climbing(problem, max_evaluations=4)

([1, 1, 0, 0], 2)

## Simulated annealing

This Simulated annealing is based on the Hill climbing method. In Simulated annealing it is allowed to select (with a low probability) a solution with a lower fitness. The probability, at step $i$, of selecting the solution with lower fitness,
$$
p(x) = exp\left({\frac{f(x) - f(b)}{T_i}}\right)
$$ 
where $x$ is the current candidate solution, $b$ is the best solution found so far and T is an array of (decreasing) temperatures.
This method reduces the risk of getting stuck in a local optimum.

It does not make sense to use Simulated annealing to solve OneMax problem because Hill climbing guarantees to find the optimal solution.

In [48]:
import math

def simulated_annealing(problem, temperature_schedule, max_evaluations):
    '''
        A simple simulated annealing algorithm for a generic problem.
        The termination condition is the number of evaluations.
    '''
    best_x = problem.random_solution() # generate a random solution
    best_fitness = problem.fitness(best_x) # evaluate the solution
    evaluations = 1 # number of solutions evaluated
    while evaluations < max_evaluations: # termination condition
        x_neighbor = problem.random_neighbor(best_x) # generate a random neighbor
        x_neighbor_fitness = problem.fitness(x_neighbor) # evaluate the neighbor
        evaluations += 1 # increase the counter
        if x_neighbor_fitness > best_fitness: # if we found a better solution
            best_x = x_neighbor.copy() # update the best solution
            best_fitness = x_neighbor_fitness # update the fitness of the best solution
        elif random.random() < math.exp((x_neighbor_fitness - best_fitness) / temperature_schedule(evaluations)):
            best_x = x_neighbor.copy()
    
    return (best_x, best_fitness)